In [ ]:
%matplotlib inline
import report
report.load("report.dat")
report.print_summary()

In [ ]:
for d in report.designs:
    report.plot_cmp_tools(d)

In [ ]:
for d in report.designs:
    report.plot_est_maxfreq(d)

In [ ]:
for d in report.designs:
    report.plot_place_route(d)